In [ ]:
# mount drive

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

# Read the ESP32 log file
with open('/content/drive/path/here', 'r') as file:
    lines = file.readlines()

# Initialize a list to hold the cleaned data
data = []

# Process each line
for line in lines:
    parts = line.strip().split(' - ')
    if len(parts) == 2:
        timestamp, log_entry = parts
        data.append({'timestamp': timestamp, 'log_entry': log_entry})

# Convert to DataFrame
df = pd.DataFrame(data)

# Save to CSV
df.to_csv('esp32_log_cleaned.csv', index=False)

print("ESP32 log processed and saved.")


In [ ]:
import pandas as pd
from datetime import datetime, timedelta

# Path to the iperf3 results file
file_path = '/content/drive/path/here'

# Read the file
with open(file_path, 'r') as file:
    lines = file.readlines()

# Initialize variables
data = []
summary_data = []
entry = {}
start_time = None

# Process each line
for line in lines:
    # Detect start time from "Running iperf3 test at ..." lines
    if line.startswith("Running iperf3 test at"):
        full_start_time = line.replace("Running iperf3 test at ", "").strip()
        # Extract only the time portion (hh:mm:ss)
        parsed_start_time = datetime.strptime(full_start_time, "%a %d %b %H:%M:%S GMT %Y")
        start_time = parsed_start_time.time()
        continue

    # Detect summary statistics rows
    if 'sender' in line or 'receiver' in line:
        parts = line.split()
        summary_entry = {
            'timestamp': parsed_start_time.strftime('%H:%M:%S'),  # Add full start time as a timestamp
            'role': 'sender' if 'sender' in line else 'receiver',
            'interval': parts[2],  # e.g., "0.00-10.00"
            'transfer': parts[4],
            'bitrate': parts[6],
            'retr': parts[8] if 'sender' in line else None,  # 'retr' exists only for the sender
        }
        summary_data.append(summary_entry)
        continue

    # Capture data lines containing iperf3 output
    if 'sec' in line and ('MBytes' in line or 'Bytes' in line) and 'Mbits/sec' in line:
        parts = line.split()
        if len(parts) >= 11:  # Full data line with cwnd
            interval = parts[2]  # e.g., "0.00-1.00"
            start_seconds = int(float(interval.split('-')[0]))  # Start of the interval in seconds
            timestamp = (datetime.combine(datetime.today(), start_time) + timedelta(seconds=start_seconds)).strftime('%H:%M:%S')

            entry.update({
                'timestamp': timestamp,
                'transfer': parts[4],
                'bitrate': parts[6],
                'retr': parts[8],
                'cwnd': parts[9]  # Isolate the numerical value from cwnd
            })
        data.append(entry)
        entry = {}

# Convert to DataFrames
df_detailed = pd.DataFrame(data)
df_summary = pd.DataFrame(summary_data)

# Save the cleaned data to CSV files
output_path_detailed = 'iperf3_results_detailed.csv'
output_path_summary = 'iperf3_results_summary.csv'
df_detailed.to_csv(output_path_detailed, index=False)
df_summary.to_csv(output_path_summary, index=False)

print(f"Detailed results saved to {output_path_detailed}.")
print(f"Summary statistics saved to {output_path_summary}.")

In [ ]:
import pandas as pd
from datetime import datetime

def process_ping_file(file_path):
    # Read the ping results file
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Initialize a list to hold the cleaned data
    data = []
    current_timestamp = None

    # Process each line
    for line in lines:
        # Detect timestamp line (e.g., "Pinging 172.20.10.6 at Sat 14 Dec 21:01:27 GMT 2024")
        if line.startswith("Pinging"):
            parts = line.split(" at ")
            if len(parts) > 1:
                timestamp_str = parts[1].strip()
                current_timestamp = datetime.strptime(timestamp_str, "%a %d %b %H:%M:%S GMT %Y")
            continue

        # Process lines with ping data (e.g., "64 bytes from 172.20.10.6: icmp_seq=1 ttl=64 time=12.0 ms")
        if "bytes from" in line:
            parts = line.split()
            seq = parts[4].split('=')[1]  # Extract sequence number
            ttl = parts[5].split('=')[1]  # Extract TTL
            time = parts[6].split('=')[1]  # Extract time (e.g., "12.0 ms")
            time_value = float(time[:-2])  # Remove "ms" and convert to float

            # Append the data with the current timestamp
            data.append({
                'timestamp': current_timestamp.strftime('%Y-%m-%d %H:%M:%S') if current_timestamp else None,
                'seq': int(seq),
                'ttl': int(ttl),
                'time_ms': time_value
            })

    return data

# File paths to the ping results
file_path_1 = '/content/drive/path/here'
file_path_2 = '/content/drive/path/here'

# Process the ping files
ping_data_1 = process_ping_file(file_path_1)
ping_data_2 = process_ping_file(file_path_2)

# Convert to DataFrame
df_ping_1 = pd.DataFrame(ping_data_1)
df_ping_2 = pd.DataFrame(ping_data_2)

# Save to CSV
output_path_1 = 'ping_base_results_cleaned.csv'
output_path_2 = 'ping_pi_results_cleaned.csv'
df_ping_1.to_csv(output_path_1, index=False)
df_ping_2.to_csv(output_path_2, index=False)

print(f"Ping results processed and saved to {output_path_1}.")
print(f"Ping results processed and saved to {output_path_2}.")
